In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/draft-guide-by-sam-vecenie/draft_guide_data.csv


In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
pd.set_option('display.max_rows', None)

## Read Data

In [4]:
df = pd.read_csv('/kaggle/input/draft-guide-by-sam-vecenie/draft_guide_data.csv')
print(df.shape)
df.head(1)

(288, 13)


,RANK,PLAYER,SCHOOL/TEAM,POS,AGE,HT,WING,TIER,STRENGTHS,WEAKNESSES,SUMMARY,TIER_DESCRIP,year
0,1,LaMelo Ball,Illawarra Hawks,G,19,6-6,6-9,5,Everything starts with Ball’s elite-level feel...,"The defense isn’t a sure thing, though, becaus...",Ball should enter the NBA as one of the most c...,All-Star Upside,2020


In [5]:
# Add Strengths and Weaknesses
df['full_text'] = df[['STRENGTHS', 'WEAKNESSES']].apply(lambda x: x[0] + ' ' + x[1], axis=1)
df.head(1)

,RANK,PLAYER,SCHOOL/TEAM,POS,AGE,HT,WING,TIER,STRENGTHS,WEAKNESSES,SUMMARY,TIER_DESCRIP,year,full_text
0,1,LaMelo Ball,Illawarra Hawks,G,19,6-6,6-9,5,Everything starts with Ball’s elite-level feel...,"The defense isn’t a sure thing, though, becaus...",Ball should enter the NBA as one of the most c...,All-Star Upside,2020,Everything starts with Ball’s elite-level feel...


In [6]:
df['TIER'] = df['TIER'] - 1

## Preprocessing

In [7]:
test = df.sample(50, random_state=42)
# test = df[df['year'] == 2023]
test.sort_values('TIER', ascending=False).head()

,RANK,PLAYER,SCHOOL/TEAM,POS,AGE,HT,WING,TIER,STRENGTHS,WEAKNESSES,SUMMARY,TIER_DESCRIP,year,full_text
140,3,Cam Whitmore,Villanova,W,18,6-6,6-9,4,"Elite intersection of size, frame, athleticism...",Whitmore has all the physical tools to be an N...,Whitmore is a pure upside play. If you buy int...,All-Star Upside,2023,"Elite intersection of size, frame, athleticism..."
60,1,Cade Cunningham,Oklahoma State,G/W,19,6-8,7-1,4,Cunningham has elite-level size as a lead crea...,"Cunningham is a good athlete, but not a great ...",Cunningham is one of the more complete prospec...,All-Star Upside,2021,Cunningham has elite-level size as a lead crea...
216,4,Jaden Ivey,Purdue,G,20,6-4,NaN,4,Was a consensus four-star recruit who was a to...,I don’t think I’d call Ivey a negative in term...,Ivey is all about how much you value athletic ...,All-Star Upside,2022,Was a consensus four-star recruit who was a to...
143,6,Jarace Walker,Houston,F,19,6-7,7-3,3,Walker has strong physical measurements for an...,How high you are on Walker probably depends on...,Walker is a fascinating player. It’s easy to i...,High-Leverage Starters,2023,Walker has strong physical measurements for an...
144,7,Taylor Hendricks,UCF,W/F,19,6-8,7-1,3,Hendricks is a good athlete with a strong fram...,"The main concern is that, for a potential top-...",I love Hendricks going to a team that is well-...,High-Leverage Starters,2023,Hendricks is a good athlete with a strong fram...


In [8]:
train = df[~df.index.isin(test.index)]
train.shape

(238, 14)

In [9]:
train.groupby('TIER')['PLAYER'].count()

TIER
0    97
1    79
2    34
3    15
4    13
Name: PLAYER, dtype: int64

In [10]:
copy_dict = {2: 2, 3: 5, 4: 5}

for i in range(2, 5):
    temp = train[train['TIER'] == i].copy()
    for j in range(copy_dict[i]):
        train = pd.concat([train, temp])
    train = train.reset_index().drop('index', axis=1)

In [11]:
train.groupby('TIER')['PLAYER'].count()

TIER
0     97
1     79
2    102
3     90
4     78
Name: PLAYER, dtype: int64

In [12]:
labels = df['TIER_DESCRIP'].unique()
id2label = {row[0]: row[1]
            for row in df[['TIER', 'TIER_DESCRIP']].drop_duplicates().values}
label2id = {row[1]: row[0]
            for row in df[['TIER', 'TIER_DESCRIP']].drop_duplicates().values}

## Preprocessing


In [13]:
import torch
from torch.nn import functional as F
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from datasets import Dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [14]:
MODEL = 'bert-base-uncased'
CONFIG = {
    "weight_decay": 0.01,
    "num_train_epochs": 15.0,
    "load_best_model_at_end": True,
    "metric_for_best_model": 'eval_loss',
}

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification\
        .from_pretrained(MODEL, 
                         id2label=id2label,
                         label2id=label2id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [16]:
model.classifier = torch.nn.Linear(
    model.classifier.in_features,
    len(labels)
)

In [17]:
def preprocess_function(examples):
    tokens = tokenizer(examples['full_text'], 
                       truncation=True, 
                       max_length=512)
    if 'TIER' in examples:
        labels_matrix = [[0.0 if examples['TIER'][i] != j else 1.0
                          for j in range(len(labels))]
                         for i in range(len(examples['TIER']))]
        return {**tokens, "labels": labels_matrix}
    else:
        return tokens

In [18]:
train_dataset = (
    Dataset.from_pandas(train[['PLAYER', 'full_text', 'TIER']])
    .map(preprocess_function, batched=True)
    .remove_columns(['TIER'])
    .shuffle(seed=42)
    .train_test_split(seed=42)
)

test_dataset = (
    Dataset.from_pandas(test[['PLAYER', 'full_text', 'TIER']])
    .map(preprocess_function, batched=True)
    .remove_columns(['TIER'])
    .shuffle(seed=42)
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
example = train_dataset['train'][0]
example['PLAYER'], example['labels']

('Onyeka Okongwu', [0.0, 0.0, 0.0, 1.0, 0.0])

## Modeling

In [20]:
from sklearn.metrics import f1_score, accuracy_score, log_loss, mean_squared_error
from sklearn.metrics import classification_report

In [21]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # replace with regression loss
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = F.cross_entropy(F.softmax(logits), labels)
        return (loss, outputs) if return_outputs else loss
    
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = F.softmax(torch.tensor(logits))
    labels_adj = [np.argmax(l) for l in labels]
    predictions_adj = [np.sum([j * p[j] for j in range(5)]) for p in predictions]
    return {"RMSE": log_loss(labels, predictions)}

In [22]:
os.environ["WANDB_DISABLED"] = "true"

torch.cuda.empty_cache()

In [23]:
run_name = f"fpell-{int(time.time())}"

print("=" * 50)
print(f"Starting run: {run_name}")
print("=" * 50)

training_args = TrainingArguments(
    run_name=run_name,
    output_dir="./results",
    save_total_limit=5,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    **CONFIG,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset["train"],
    eval_dataset=train_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting run: fpell-1692202484


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_22/1591327229.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = F.cross_entropy(F.softmax(logits), labels)


Step,Training Loss,Validation Loss,Rmse
50,1.615000,1.614447,1.759654
100,1.605700,1.612612,1.713267
150,1.605400,1.606736,1.683304
200,1.607900,1.608927,1.718789
250,1.607700,1.608682,1.687486
300,1.605800,1.608525,1.695022
350,1.601500,1.610349,1.710182
400,1.605600,1.613515,1.722185
450,1.602300,1.612231,1.721780
500,1.601800,1.611534,1.716529


/tmp/ipykernel_22/1591327229.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = F.softmax(torch.tensor(logits))
/tmp/ipykernel_22/1591327229.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = F.cross_entropy(F.softmax(logits), labels)
/tmp/ipykernel_22/1591327229.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = F.softmax(torch.tensor(logits))
/tmp/ipykernel_22/1591327229.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = F.cross_entropy(F.softmax(logits), labels)
/tmp/ipykernel_22/1591327229.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = F

TrainOutput(global_step=630, training_loss=1.6055489434136285, metrics={'train_runtime': 309.7607, 'train_samples_per_second': 16.174, 'train_steps_per_second': 2.034, 'total_flos': 1318221893744640.0, 'train_loss': 1.6055489434136285, 'epoch': 15.0})

In [24]:
temp = trainer.predict(train_dataset['train'])
log_loss(temp[1], F.softmax(torch.tensor(temp[0])))

/tmp/ipykernel_22/1591327229.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = F.cross_entropy(F.softmax(logits), labels)


/tmp/ipykernel_22/1591327229.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = F.softmax(torch.tensor(logits))
/tmp/ipykernel_22/995027616.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  log_loss(temp[1], F.softmax(torch.tensor(temp[0])))


1.6870752774152697

In [25]:
temp = trainer.predict(train_dataset['test'])
log_loss(temp[1], F.softmax(torch.tensor(temp[0])))

/tmp/ipykernel_22/1591327229.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = F.cross_entropy(F.softmax(logits), labels)


/tmp/ipykernel_22/1591327229.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = F.softmax(torch.tensor(logits))
/tmp/ipykernel_22/3491228456.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  log_loss(temp[1], F.softmax(torch.tensor(temp[0])))


1.7165288824457254

In [26]:
temp = trainer.predict(test_dataset)
log_loss(temp[1], F.softmax(torch.tensor(temp[0])))

/tmp/ipykernel_22/1591327229.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = F.cross_entropy(F.softmax(logits), labels)


/tmp/ipykernel_22/1591327229.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = F.softmax(torch.tensor(logits))
/tmp/ipykernel_22/1060597350.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  log_loss(temp[1], F.softmax(torch.tensor(temp[0])))


1.790636463984143

In [27]:
data = []

for i in range(len(test_dataset)):
    data.append([test_dataset[i]['PLAYER'], 
                 temp[1][i],
                 F.softmax(torch.tensor(temp[0][i]))])
    
res = pd.DataFrame(data, columns=['PLAYER', 'TIER', 'pred'])
res['label'] = res['TIER'].apply(lambda x: np.argmax(x))
res['pred_label'] = res['pred'].apply(lambda x: np.sum([i*x[i] for i in range(5)]))
res['diff'] = np.abs(res['label'] - res['pred_label'])

res['log_loss'] = res[['TIER', 'pred']].apply(lambda x: log_loss(x[0], x[1]), 
                                              axis=1)
res.sort_values('diff', ascending=False)


/tmp/ipykernel_22/456294528.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.tensor(temp[0][i]))])


,PLAYER,TIER,pred,label,pred_label,diff,log_loss
29,Jaden Ivey,"[0.0, 0.0, 0.0, 0.0, 1.0]","[tensor(0.2154), tensor(0.1152), tensor(0.4066...",4,1.811119,2.188881,0.686615
24,Cam Whitmore,"[0.0, 0.0, 0.0, 0.0, 1.0]","[tensor(0.2152), tensor(0.1153), tensor(0.4069...",4,1.811261,2.188739,0.686868
18,Cade Cunningham,"[0.0, 0.0, 0.0, 0.0, 1.0]","[tensor(0.2151), tensor(0.1153), tensor(0.4070...",4,1.811294,2.188706,0.686932
2,Jared Rhoden,"[1.0, 0.0, 0.0, 0.0, 0.0]","[tensor(0.2153), tensor(0.1153), tensor(0.4067...",0,1.810875,1.810875,0.492736
20,Julian Strawther,"[1.0, 0.0, 0.0, 0.0, 0.0]","[tensor(0.2151), tensor(0.1153), tensor(0.4072...",0,1.810866,1.810866,0.493002
16,Omari Moore,"[1.0, 0.0, 0.0, 0.0, 0.0]","[tensor(0.2151), tensor(0.1153), tensor(0.4074...",0,1.810819,1.810819,0.493107
37,Juhann Begarin,"[1.0, 0.0, 0.0, 0.0, 0.0]","[tensor(0.2152), tensor(0.1153), tensor(0.4071...",0,1.810817,1.810817,0.492926
11,Adama Sanogo,"[1.0, 0.0, 0.0, 0.0, 0.0]","[tensor(0.2152), tensor(0.1153), tensor(0.4071...",0,1.810815,1.810815,0.492942
41,Luka Garza,"[1.0, 0.0, 0.0, 0.0, 0.0]","[tensor(0.2152), tensor(0.1153), tensor(0.4070...",0,1.810796,1.810796,0.492860
34,Carlik Jones,"[1.0, 0.0, 0.0, 0.0, 0.0]","[tensor(0.2153), tensor(0.1153), tensor(0.4069...",0,1.810788,1.810788,0.492813
